In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
import datetime
import import_ipynb
from tensorboard.plugins.hparams import api as hp
from dataloader import get_LFW_X_df, get_y_df, data_prep

2023-11-19 11:48:10.340115: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-19 11:48:10.387143: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-19 11:48:10.387797: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-19 11:48:11.086989: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


importing Jupyter notebook from dataloader.ipynb
Tensorflow version: 2.12.0


In [3]:
rm -rf ./logs/

In [4]:
csv_filepath = "../../data/csv/plant_data.csv"

In [5]:
X = get_LFW_X_df(csv_filepath)
y = get_y_df(csv_filepath, 'LFW_g')

In [6]:
HP_NUM_UNITS1 = hp.HParam("num_units", hp.Discrete([16, 32, 64, 128]))
HP_NUM_UNITS2 = hp.HParam("num_units", hp.Discrete([16, 32, 64, 128]))


HP_DROPOUT1 = hp.HParam("dropout", hp.RealInterval(0.1, 0.2))
HP_DROPOUT2 = hp.HParam("dropout", hp.RealInterval(0.1, 0.2))

HP_OPTIMIZER = hp.HParam("optimizer", hp.Discrete(['adam', 'sgd']))

HP_ACTIVATION1 = hp.HParam("activation", hp.Discrete(['linear', 'relu', 'sigmoid']))
HP_ACTIVATION2 = hp.HParam("activation", hp.Discrete(['linear', 'relu', 'sigmoid']))


METRIC_ACCURACY = 'accuracy'

In [7]:
with tf.summary.create_file_writer('../../logs/hyperparam_tuning/repression/LFW').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_DROPOUT1, HP_DROPOUT2, HP_OPTIMIZER, HP_ACTIVATION1, HP_ACTIVATION2],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name="Test Accuracy")]
    )

2023-11-19 11:48:12.284792: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-19 11:48:12.303830: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [8]:
X_train, X_test, y_train, y_test = data_prep(X, y)

In [9]:
def LFW_model(hparams):
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(hparams[HP_NUM_UNITS1], hparams[HP_ACTIVATION1], name='layers_input', input_shape = (11,),),
            tf.keras.layers.Dropout(hparams[HP_DROPOUT1]),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(hparams[HP_NUM_UNITS2], hparams[HP_ACTIVATION2], name='layers_dense2'),
            tf.keras.layers.Dropout(HP_DROPOUT2),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(1, name='layers_dense3')
        ])

        model.compile (
            optimizer=hparams[HP_OPTIMIZER],
            loss='mae',
            metrics=['accuracy'],
        )

        model.fit (
            X_train,
            y_train,
            epochs=100,
            validation_split=0.2,
            callbacks=[
                tf.keras.callbacks.TensorBoard(log_dir=log_dir),
                hp.KerasCallback(log_dir, hparams)
            ]
        )

        accuracy = model.evaluate(X_test, y_test)

        

In [10]:
log_dir = "../../logs/hyperparam_tuning/regression/LFW/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [11]:
def run_model(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        accuracy = LFW_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [13]:
session_num = 0

for num_units1 in HP_NUM_UNITS1.domain.values:
    for num_units2 in HP_NUM_UNITS2.domain.values:
        for droput1 in (HP_DROPOUT1.domain.min_value, HP_DROPOUT1.domain.max_value):
            for droput2 in (HP_DROPOUT2.domain.min_value, HP_DROPOUT2.domain.max_value):
                for activation1 in HP_ACTIVATION1.domain.values:
                    for activation2 in HP_ACTIVATION2.domain.values:
                        for optimizer in HP_OPTIMIZER.domain.values:
                            hparams = {
                                HP_ACTIVATION1: activation1,
                                HP_ACTIVATION2: activation2,
                                HP_DROPOUT1: droput1,
                                HP_DROPOUT2: droput2,
                                HP_NUM_UNITS1: num_units1,
                                HP_NUM_UNITS2: num_units2,
                                HP_OPTIMIZER: optimizer
                            }

                            run_name = "run-%d" % session_num
                            print('--- Starting trial: %s' % run_name)
                            print({h.name: hparams[h] for h in hparams})
                            run_model(log_dir + run_name, hparams)
                            session_num += 1

--- Starting trial: run-0
{'activation': 'linear', 'dropout': 0.1, 'num_units': 16, 'optimizer': 'adam'}


ValueError: multiple values specified for hparam 'dropout'